In [65]:
from pathlib import Path
import numpy as np
import pandas as pd
import xarray as xr
import rioxarray as rxa0

from rasterio.enums import Resampling

import matplotlib.pyplot as plt

In [114]:
model_dir = Path('~/scratch/coherence/model').expanduser()
dss = {fp: xr.open_dataset(fp, decode_coords="all") for fp in Path('~/scratch/coherence/uavsar').expanduser().glob('*_full.nc')}

model_dic = {20:'uticam', 15: 'rockmt', 6: 'fraser', 13: 'peeler', 9: 'irnton', 8: 'grmesa', 4: 'alamos', 19: 'stlake', \
            10: 'lowman', 17: 'silver', 3: 'sierra', 0: 'donner', 1: 'dorado'}

model_dic = {v: k for k, v in model_dic.items()}

In [115]:
for fp, ds in dss.items():
    stem = fp.stem.replace('_full', '')
    for var, var_name in zip(['snod', 'smlt', 'swed'], ['model_sd', 'model_melt', 'model_swe']):
        model_fps = list(model_dir.rglob(f'*{var}.nc'))
        model_fp = [fp for fp in model_fps if model_dic[stem] == int(fp.parent.stem)]
        assert len(model_fp) == 1
        model_fp == model_fp[0]
        model = xr.open_dataset(model_fp[0], decode_coords="all")
        model = model.rename({'date':'model_time'}).rio.reproject_match(ds, resampling = Resampling.bilinear) # .rio.write_crs('EPSG:4326')
        ds[var_name] = model[var]
    ds.drop_vars('band').to_netcdf(fp.parent.joinpath(stem + '_v2.nc'))

MemoryError: Unable to allocate 52.0 GiB for an array with shape (4, 12, 12, 2, 3223, 3758) and data type float32

: 

In [113]:
ds

<xarray.Dataset> Size: 6GB
Dimensions:      (pol: 4, time2: 1, time1: 1, heading: 1, y: 1531, x: 946,
                  model_time: 363)
Coordinates:
  * pol          (pol) <U2 32B 'HH' 'HV' 'VH' 'VV'
  * time2        (time2) datetime64[ns] 8B 2020-03-12
  * time1        (time1) datetime64[ns] 8B 2020-02-21
  * heading      (heading) int64 8B 347
  * x            (x) float64 8kB -116.9 -116.9 -116.9 ... -116.6 -116.6 -116.6
  * y            (y) float64 12kB 43.37 43.37 43.37 43.37 ... 42.95 42.95 42.95
    spatial_ref  int64 8B ...
    band         int64 8B ...
  * model_time   (model_time) datetime64[ns] 3kB 2019-11-01 ... 2021-04-30
Data variables:
    cor          (pol, time2, time1, heading, y, x) float32 23MB ...
    inc          (heading, y, x) float32 6MB ...
    dem          (y, x) float32 6MB ...
    vh           (y, x) float32 6MB ...
    tree_perc    (y, x) float32 6MB ...
    land_cover   (y, x) int64 12MB ...
    model_sd     (model_time, y, x) float32 2GB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    model_melt   (model_time, y, x) float32 2GB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    model_swe    (model_time, y, x) float32 2GB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0